In [1]:
import os
os.chdir('..')

In [2]:
import pprint

from kernel.term import BoolVars, Eq, Implies, Not, false, Term
from kernel.proofterm import ProofTerm
from kernel import theory
from kernel.report import ProofReport
from logic import basic
from logic import logic
from logic.conv import rewr_conv
from prover import tseitin
from prover import sat
from syntax.settings import settings

basic.load_theory('sat')
settings.unicode = True
pp = pprint.PrettyPrinter()

## Proof generation for DPLL

In this section, we combine together the work in the previous three sections, and implement a full verified solver for propositional formulas based on SAT solving. The overall procedure is: given a propositional formula $F$ which we want to prove,

1. Convert $\neg F$ to CNF form using Tseitin encoding, generating a theorem guaranteeing the correctness of the encoding.

2. Use the DPLL algorithm to check the unsatisfiability of $\neg F$ in CNF form. Return the proof as a chain of resolutions.

3. Assemble the proof of unsatisfiability of the formula from the chain of resolutions.

4. Combine with the theorem guaranteeing the encoding to give a proof of the original formula.

We start from a rather simple propositional formula, problem #1 from the problem list in "Seventy-five problems for testing automatic theorem provers:

In [3]:
p, q = BoolVars('p q')

F = Eq(Implies(p, q), Implies(Not(q), Not(p)))

Using Tseitin encoding, the formular $\neg F$ is translated as follows:

In [4]:
encode_pt = tseitin.encode(Not(F))
print(encode_pt)

cnf = tseitin.convert_cnf(encode_pt.prop)
pp.pprint(cnf)

ProofTerm(x1 ⟷ p, x2 ⟷ q, x3 ⟷ ¬x1, x4 ⟷ ¬x2, x8 ⟷ ¬x7, x5 ⟷ (x1 ⟶ x2), x6 ⟷ (x4 ⟶ x3), x7 ⟷ x5 ⟷ x6, ¬((p ⟶ q) ⟷ (¬q ⟶ ¬p)) ⊢ x8 ∧ (x1 ∨ x5) ∧ (x3 ∨ x1) ∧ (x4 ∨ x2) ∧ (x4 ∨ x6) ∧ (x8 ∨ x7) ∧ (¬x2 ∨ x5) ∧ (¬x3 ∨ x6) ∧ (x7 ∨ x5 ∨ x6) ∧ (¬x3 ∨ ¬x1) ∧ (¬x4 ∨ ¬x2) ∧ (¬x8 ∨ ¬x7) ∧ (x7 ∨ ¬x5 ∨ ¬x6) ∧ (¬x5 ∨ ¬x1 ∨ x2) ∧ (¬x6 ∨ ¬x4 ∨ x3) ∧ (¬x7 ∨ x5 ∨ ¬x6) ∧ (¬x7 ∨ ¬x5 ∨ x6))
[[('x8', True)],
 [('x1', True), ('x5', True)],
 [('x3', True), ('x1', True)],
 [('x4', True), ('x2', True)],
 [('x4', True), ('x6', True)],
 [('x8', True), ('x7', True)],
 [('x2', False), ('x5', True)],
 [('x3', False), ('x6', True)],
 [('x7', True), ('x5', True), ('x6', True)],
 [('x3', False), ('x1', False)],
 [('x4', False), ('x2', False)],
 [('x8', False), ('x7', False)],
 [('x7', True), ('x5', False), ('x6', False)],
 [('x5', False), ('x1', False), ('x2', True)],
 [('x6', False), ('x4', False), ('x3', True)],
 [('x7', False), ('x5', True), ('x6', False)],
 [('x7', False), ('x5', False), ('x6', True)]]


This is a CNF formula consisting of 8 variables and 17 clauses. The result of DPLL on this formula is as follows:

In [5]:
res, proof = sat.solve_cnf(cnf)
print(res, proof)

unsatisfiable {17: [14, 9, 1, 12, 11, 0], 18: [13, 8, 10, 2, 7, 11, 0, 17], 19: [12, 11, 0, 4, 18, 6, 3, 18]}


As expected, the formula is unsatisfiable. The proof returned consists of three chains of resolutions. To construct proofs for these, we first obtain a list of assumptions for the inital clauses.

In [6]:
clause_pts = [ProofTerm.assume(clause) for clause in encode_pt.prop.strip_conj()]
pp.pprint(clause_pts)

[ProofTerm(x8 ⊢ x8),
 ProofTerm(x1 ∨ x5 ⊢ x1 ∨ x5),
 ProofTerm(x3 ∨ x1 ⊢ x3 ∨ x1),
 ProofTerm(x4 ∨ x2 ⊢ x4 ∨ x2),
 ProofTerm(x4 ∨ x6 ⊢ x4 ∨ x6),
 ProofTerm(x8 ∨ x7 ⊢ x8 ∨ x7),
 ProofTerm(¬x2 ∨ x5 ⊢ ¬x2 ∨ x5),
 ProofTerm(¬x3 ∨ x6 ⊢ ¬x3 ∨ x6),
 ProofTerm(x7 ∨ x5 ∨ x6 ⊢ x7 ∨ x5 ∨ x6),
 ProofTerm(¬x3 ∨ ¬x1 ⊢ ¬x3 ∨ ¬x1),
 ProofTerm(¬x4 ∨ ¬x2 ⊢ ¬x4 ∨ ¬x2),
 ProofTerm(¬x8 ∨ ¬x7 ⊢ ¬x8 ∨ ¬x7),
 ProofTerm(x7 ∨ ¬x5 ∨ ¬x6 ⊢ x7 ∨ ¬x5 ∨ ¬x6),
 ProofTerm(¬x5 ∨ ¬x1 ∨ x2 ⊢ ¬x5 ∨ ¬x1 ∨ x2),
 ProofTerm(¬x6 ∨ ¬x4 ∨ x3 ⊢ ¬x6 ∨ ¬x4 ∨ x3),
 ProofTerm(¬x7 ∨ x5 ∨ ¬x6 ⊢ ¬x7 ∨ x5 ∨ ¬x6),
 ProofTerm(¬x7 ∨ ¬x5 ∨ x6 ⊢ ¬x7 ∨ ¬x5 ∨ x6)]


Now, for the first proof, we can construct the resolution according to the numbers:

In [7]:
lst = proof[17]
pt = clause_pts[lst[0]]
print('Initial clause:', pt.prop)
for step in lst[1:]:
    pt = logic.resolution(pt, clause_pts[step])
    print('Resolution with %s to get %s' % (clause_pts[step].prop, pt.prop))

Initial clause: ¬x6 ∨ ¬x4 ∨ x3
Resolution with ¬x3 ∨ ¬x1 to get ¬x1 ∨ ¬x4 ∨ ¬x6
Resolution with x1 ∨ x5 to get x5 ∨ ¬x4 ∨ ¬x6
Resolution with x7 ∨ ¬x5 ∨ ¬x6 to get x7 ∨ ¬x4 ∨ ¬x6
Resolution with ¬x8 ∨ ¬x7 to get ¬x4 ∨ ¬x6 ∨ ¬x8
Resolution with x8 to get ¬x4 ∨ ¬x6


The result $\neg x_3$ is then added to the list of clauses, and we can continue with the next item in the proof. The last step of the proof should give us a contradiction. The full code is:

In [8]:
for new_id in sorted(proof.keys()):
    steps = proof[new_id]
    pt = clause_pts[steps[0]]
    for step in steps[1:]:
        pt = logic.resolution(pt, clause_pts[step])
    clause_pts.append(pt)
    
contra_pt = clause_pts[-1]
assert contra_pt.prop == false

We have now obtained a proof term showing a contradiction from the clauses of the CNF. This can be easily transformed into a theorem showing a contradiction from the CNF itself:

In [9]:
pt1, pt2 = encode_pt, contra_pt
while pt1.prop.is_conj():
    pt_left = logic.apply_theorem('conjD1', pt1)
    pt2 = pt2.implies_intr(pt_left.prop).implies_elim(pt_left)  # remove one clause from assumption
    pt1 = logic.apply_theorem('conjD2', pt1)
pt2 = pt2.implies_intr(pt1.prop).implies_elim(pt1)  # remove last clause from assumption
print(pt2)

ProofTerm(x1 ⟷ p, x2 ⟷ q, x3 ⟷ ¬x1, x4 ⟷ ¬x2, x8 ⟷ ¬x7, x5 ⟷ (x1 ⟶ x2), x6 ⟷ (x4 ⟶ x3), x7 ⟷ x5 ⟷ x6, ¬((p ⟶ q) ⟷ (¬q ⟶ ¬p)) ⊢ false)


Finally, we need to remove the dependence on each of the equalities. This need to be done in the reverse order of variables, from $x_8$ back to $x_1$. We first obtain the list of equalities in reverse order.

In [10]:
eqs = [t for t in pt2.hyps if t.is_equals()]
eqs = list(reversed(sorted(eqs, key=lambda t: int(t.lhs().name[1:]))))

Next, we remove one equality at a time. This works by introducing the equality into the proposition, forall the new variable, then substitute the right side. Let's try it for one step:

In [11]:
eq = eqs[0]
print(pt2.implies_intr(eq).forall_intr(eq.lhs).forall_elim(eq.rhs)
         .implies_elim(ProofTerm.reflexive(eq.rhs)))

ProofTerm(x1 ⟷ p, x2 ⟷ q, x3 ⟷ ¬x1, x4 ⟷ ¬x2, x5 ⟷ (x1 ⟶ x2), x6 ⟷ (x4 ⟶ x3), x7 ⟷ x5 ⟷ x6, ¬((p ⟶ q) ⟷ (¬q ⟶ ¬p)) ⊢ false)


Iterative over all of eqs, we get:

In [12]:
for eq in eqs:
    pt2 = pt2.implies_intr(eq).forall_intr(eq.lhs).forall_elim(eq.rhs) \
             .implies_elim(ProofTerm.reflexive(eq.rhs))
print(pt2)

ProofTerm(¬((p ⟶ q) ⟷ (¬q ⟶ ¬p)) ⊢ false)


Finally, we finish by applying `negI` and `double_neg`:

In [13]:
logic.apply_theorem('negI', pt2.implies_intr(pt2.hyps[0])).on_prop(rewr_conv('double_neg'))

ProofTerm(⊢ (p ⟶ q) ⟷ (¬q ⟶ ¬p))

The full code is below:

In [14]:
def solve_cnf(F):
    encode_pt = tseitin.encode(Not(F))
    cnf = tseitin.convert_cnf(encode_pt.prop)
    res, proof = sat.solve_cnf(cnf)
    assert res == 'unsatisfiable', 'solve_cnf: statement is not provable'
    
    # Perform the resolution steps
    clause_pts = [ProofTerm.assume(clause) for clause in encode_pt.prop.strip_conj()]
    for new_id in sorted(proof.keys()):
        steps = proof[new_id]
        pt = clause_pts[steps[0]]
        for step in steps[1:]:
            pt = logic.resolution(pt, clause_pts[step])
        clause_pts.append(pt)

    contra_pt = clause_pts[-1]
    assert contra_pt.prop == false
    
    # Show contradiction from ~F and definitions of new variables
    pt1, pt2 = encode_pt, contra_pt
    while pt1.prop.is_conj():
        pt_left = logic.apply_theorem('conjD1', pt1)
        pt2 = pt2.implies_intr(pt_left.prop).implies_elim(pt_left)  # remove one clause from assumption
        pt1 = logic.apply_theorem('conjD2', pt1)
    pt2 = pt2.implies_intr(pt1.prop).implies_elim(pt1)  # remove last clause from assumption

    # Clear definition of new variables from antecedent
    eqs = [t for t in pt2.hyps if t.is_equals()]
    eqs = list(reversed(sorted(eqs, key=lambda t: int(t.lhs().name[1:]))))

    for eq in eqs:
        pt2 = pt2.implies_intr(eq).forall_intr(eq.lhs).forall_elim(eq.rhs) \
                 .implies_elim(ProofTerm.reflexive(eq.rhs))

    return logic.apply_theorem('negI', pt2.implies_intr(pt2.hyps[0])).on_prop(rewr_conv('double_neg'))

In [15]:
solve_cnf(F)

ProofTerm(⊢ (p ⟶ q) ⟷ (¬q ⟶ ¬p))

We can test this on some other examples from Pelletier's problem list:

In [16]:
# Problem 2
solve_cnf(Term("~~p <--> p"))

ProofTerm(⊢ ¬¬p ⟷ p)

In [17]:
# Problem 4
solve_cnf(Term("(~p --> q) <--> (~q --> p)"))

ProofTerm(⊢ (¬p ⟶ q) ⟷ (¬q ⟶ p))

In [18]:
# Problem 5
solve_cnf(Term("((p | q) --> (p | r)) --> (p | (q --> r))"))

ProofTerm(⊢ (p ∨ q ⟶ p ∨ r) ⟶ p ∨ (q ⟶ r))

In [19]:
# Problem 8
solve_cnf(Term("((p --> q) --> p) --> p"))

ProofTerm(⊢ ((p ⟶ q) ⟶ p) ⟶ p)

In [20]:
# Problem 12
p12_pt = solve_cnf(Term("((p <--> q) <--> r) <--> (p <--> (q <--> r))"))
print(p12_pt)

ProofTerm(⊢ ((p ⟷ q) ⟷ r) ⟷ p ⟷ q ⟷ r)


The last example is takes a bit long, let's see statistics from checking the proof (evaluation of this cell takes some time).

In [21]:
rpt = ProofReport()
theory.check_proof(p12_pt.export(), rpt=rpt)
print(rpt)

Steps: 11519
  Theorems:  3470
  Primitive: 8049
  Macro:     0
Theorems applied: resolution, conjD2, trivial, disjE2, syllogism, resolution_right, resolution_left, double_neg, disjI2, negE, negI, conjI, encode_eq, encode_not, conjD1, disjI1
Macros evaluated: 
Macros expanded: resolution, imp_disj, apply_theorem, apply_theorem_for, imp_conj
Gaps: []
